<a href="https://colab.research.google.com/github/jwb4335/nlp/blob/master/hw2_part1/hw2part1_johnbarry_20200120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 2 part 1
## CS 590.07
## John Barry





## Load in simlex999 data from github

In [0]:
import pandas as pd
from random import randint



In [0]:
url = 'https://raw.githubusercontent.com/jwb4335/nlp/master/hw2_part1/Sim_Lex999.txt'
simlex = pd.read_csv(url,sep = ':')
simlex.columns = ['word1','word2','SimLex999','POS1','POS2']



```
# This is formatted as code
```

Let's get three random rows

In [4]:
randints = [randint(0,len(simlex)), randint(0,len(simlex)),randint(0,len(simlex))]

simlex_rand = simlex.iloc[randints,:]
simlex_rand



,word1,word2,SimLex999,POS1,POS2
934,reduce,shrink,8.02,v,v
787,kill,destroy,5.90,v,v
107,fresh,wide,0.40,a,a


## **Universal sentence encoder** 





In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [8]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: breakfast
Embedding size: 512
Embedding: [-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904, ...]



# CBOW or word2index

Download the data

In [9]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt"
#!cat reviews.txt | head

--2020-01-27 12:53:29--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt’

reviews.txt         100%[===================>]  32.12M  41.2MB/s    in 0.8s    

2020-01-27 12:53:33 (41.2 MB/s) - ‘reviews.txt’ saved [33678267/33678267]



In [10]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt"
#!cat labels.txt | head -15

--2020-01-27 12:53:34--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.05s   

2020-01-27 12:53:36 (4.31 MB/s) - ‘labels.txt’ saved [225000/225000]



Predicting Movie Reviews

In [0]:
import sys

f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

Initialize Weights

In [0]:
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha, iterations = (0.01, 10)
hidden_size = 200

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1


#print (weights_0_1)

Run the Model

In [13]:


correct,total = (0,0)

for iter in range(iterations):
    
    # train on first 24,000
    for i in range(len(input_dataset)-1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax

        layer_2_delta = layer_2 - y # compare pred with truth
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                             +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()
    
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    
    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))




Iter:0 Progress:95.99% Training Accuracy:0.8309583333333334%
Iter:1 Progress:95.99% Training Accuracy:0.8658541666666667%
Iter:2 Progress:95.99% Training Accuracy:0.8840833333333333%
Iter:3 Progress:95.99% Training Accuracy:0.89725%
Iter:4 Progress:95.99% Training Accuracy:0.9074833333333333%
Iter:5 Progress:95.99% Training Accuracy:0.9159861111111111%
Iter:6 Progress:95.99% Training Accuracy:0.9231369047619048%
Iter:7 Progress:95.99% Training Accuracy:0.929375%
Iter:8 Progress:95.99% Training Accuracy:0.9350462962962963%
Iter:9 Progress:95.99% Training Accuracy:0.9401%
Test Accuracy:0.831


Let's get the vector for a word

In [0]:
target = "dentist"
target_index = word2index[target]
#weights_0_1[target_index]

# Word2vec

In [15]:
!wget http://mattmahoney.net/dc/text8.zip -O text8.zip

--2020-01-27 12:55:04--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M   344KB/s    in 93s     

2020-01-27 12:56:38 (328 KB/s) - ‘text8.zip’ saved [31344016/31344016]



In [16]:
!unzip -o text8.zip

Archive:  text8.zip
  inflating: text8                   


In [0]:
!pip install --quiet gensim  

In [0]:
from gensim.models import word2vec

In [19]:
# https://github.com/dav/word2vec/blob/master/scripts/create-text8-vector-data.sh#L16
model = word2vec.Word2Vec(corpus_file='text8',size=100, window=5, min_count=1, workers=10)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Get the similarity scores of the three embeddings

In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
pd.options.mode.chained_assignment = None  # default='warn'
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('Embedding this pair from Simlex_999: "{}" and "{}"'.format(word1, word2))
  print('Running universal sentence encoder on "{}"'.format(word1))
  messages = [word1]
  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
      #print("Message: {}".format(messages[i]))
      #print("Embedding size: {}".format(len(message_embedding)))
      message_embedding_snippet = ", ".join(
          (str(x) for x in message_embedding[:3]))
      #print("Embedding: [{}, ...]".format(message_embedding_snippet))

  vector1 = message_embedding
  vector1 = np.asarray(vector1)
  vector1 = vector1.reshape(1,512)

  print('Running universal sentence encoder on "{}"'.format(word2))
  messages = [word2]

  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    #print("Message: {}".format(messages[i]))
    #print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))

  vector2 = message_embedding
  vector2 = np.asarray(vector2)
  vector2 = vector2.reshape(1,512)

  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from Universal sentence encoder for {} and {} = {}\n'.format(word1,word2,cos_sim) )
  simlex_rand.loc[simlex_rand.index == num,'USE Cosine Sim'] = cos_sim

Embedding this pair from Simlex_999: "reduce" and "shrink"
Running universal sentence encoder on "reduce"
Running universal sentence encoder on "shrink"
Cosine similarity from Universal sentence encoder for reduce and shrink = [[0.59188971]]

Embedding this pair from Simlex_999: "kill" and "destroy"
Running universal sentence encoder on "kill"
Running universal sentence encoder on "destroy"
Cosine similarity from Universal sentence encoder for kill and destroy = [[0.74396529]]

Embedding this pair from Simlex_999: "fresh" and "wide"
Running universal sentence encoder on "fresh"
Running universal sentence encoder on "wide"
Cosine similarity from Universal sentence encoder for fresh and wide = [[0.45424075]]



Embedding method 2

In [22]:
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('running word2index on the pair {} and {}'.format(word1,word2))
  target_index1 = word2index[word1]
  target_index2 = word2index[word2]
  vector1 = weights_0_1[target_index1]
  vector1 = vector1.reshape(1,200)
  vector2 = weights_0_1[target_index2]
  vector2 = vector2.reshape(1,200)
  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from word2index for {} and {} = {}\n'.format(word1,word2,cos_sim))
  simlex_rand.loc[simlex_rand.index == num,'word2index Cosine Sim'] = cos_sim


running word2index on the pair reduce and shrink
Cosine similarity from word2index for reduce and shrink = [[0.0400988]]

running word2index on the pair kill and destroy
Cosine similarity from word2index for kill and destroy = [[0.45333789]]

running word2index on the pair fresh and wide
Cosine similarity from word2index for fresh and wide = [[-0.09060798]]



Embedding method word2vec



In [23]:
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('running word2vec on the pair {} and {}'.format(word1,word2))
  vector1 = model.wv[word1]
  vector1 = vector1.reshape(1,100)
  vector2 = model.wv[word2]
  vector2 = vector2.reshape(1,100)
  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from word2vec for {} and {} = {}\n'.format(word1,word2,cos_sim))
  simlex_rand.loc[simlex_rand.index == num,'word2vec Cosine Sim'] = cos_sim



running word2vec on the pair reduce and shrink
Cosine similarity from word2vec for reduce and shrink = [[-0.18979317]]

running word2vec on the pair kill and destroy
Cosine similarity from word2vec for kill and destroy = [[0.9925319]]

running word2vec on the pair fresh and wide
Cosine similarity from word2vec for fresh and wide = [[0.9954476]]



In [24]:
simlex_rand_out = simlex_rand[['word1','word2','SimLex999', 'USE Cosine Sim',
       'word2index Cosine Sim', 'word2vec Cosine Sim']]
simlex_rand_out.columns = ['word1','word2','SimLex999', 'USE',
       'CBOW', 'word2vec']
simlex_rand_out = round(simlex_rand_out,4)
print('The cosine similarities from the 3 word embedding methods can be seen below.\nThe similarity scores are not close to the Simlex999 scores\n word2vec probably does best.')
print(simlex_rand_out)

The cosine similarities from the 3 word embedding methods can be seen below.
The similarity scores are not close to the Simlex999 scores
 word2vec probably does best.
      word1    word2  SimLex999     USE    CBOW  word2vec
934  reduce   shrink       8.02  0.5919  0.0401   -0.1898
787    kill  destroy       5.90  0.7440  0.4533    0.9925
107   fresh     wide       0.40  0.4542 -0.0906    0.9954


# Part 2, generate three .csv files with similarity scores of all words ending in 4, using my three embedded models.

In [25]:
## Unique id is jwb74, but python starts on zero, so I need to minus 1 to get the right words
unique_id = 4 
index = np.arange(4-1,1000,10)
simlex_out = simlex.iloc[index,:].reset_index(drop=True)[['word1','word2','SimLex999']]
simlex_out.columns = ['word1','word2','Sim_lex_Similarity']
simlex_out

,word1,word2,Sim_lex_Similarity
0,happy,cheerful,9.55
1,bad,terrible,7.78
2,dumb,foolish,6.67
3,weird,normal,0.72
4,nice,cruel,0.67
...,...,...,...
95,try,think,2.62
96,leave,appear,0.97
97,put,hang,3.00
98,say,verify,4.90


In [0]:
def printbold(string):
  class bold:
    BOLD = '\033[1m'
    ENDC = '\033[0m'
  print(f"{bold.BOLD}{string}{bold.ENDC}")

In [0]:
import copy
from google.colab import files


## Embedding method 2

In [28]:
simlex_out_CBOW = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  printbold('word pair {}'.format(num))
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  print('running word2index on the pair {} and {}'.format(word1,word2))
  try:
    target_index1 = word2index[word1]
    target_index2 = word2index[word2]
    vector1 = weights_0_1[target_index1]
    vector1 = vector1.reshape(1,200)
    vector2 = weights_0_1[target_index2]
    vector2 = vector2.reshape(1,200)
    cos_sim = cosine_similarity(vector1, vector2)
    print('Cosine similarity from word2index for {} and {} = {}'.format(word1,word2,cos_sim))
    simlex_out_CBOW.loc[simlex_out_CBOW.index == num,'your_Similarity'] = cos_sim
  except:
    print('skipping the pair {} and {}'.format(word1,word2))


word pair 0
running word2index on the pair happy and cheerful
Cosine similarity from word2index for happy and cheerful = [[-0.36416571]]
word pair 1
running word2index on the pair bad and terrible
Cosine similarity from word2index for bad and terrible = [[0.85411501]]
word pair 2
running word2index on the pair dumb and foolish
Cosine similarity from word2index for dumb and foolish = [[0.12147803]]
word pair 3
running word2index on the pair weird and normal
Cosine similarity from word2index for weird and normal = [[0.33807444]]
word pair 4
running word2index on the pair nice and cruel
Cosine similarity from word2index for nice and cruel = [[-0.16975908]]
word pair 5
running word2index on the pair bad and terrific
Cosine similarity from word2index for bad and terrific = [[-0.69049711]]
word pair 6
running word2index on the pair harsh and cruel
Cosine similarity from word2index for harsh and cruel = [[-0.17936353]]
word pair 7
running word2index on the pair bad and guilty
Cosine similarit

In [29]:
simlex_out_CBOW.to_csv('CBOW.csv',sep='\t')
files.download('CBOW.csv')

MessageError: ignored

## Embedding method 3

In [0]:
simlex_out_word2vec = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  printbold('word pair {}'.format(num))
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  try:
    print('running word2vec on the pair {} and {}'.format(word1,word2))
    vector1 = model.wv[word1]
    vector1 = vector1.reshape(1,100)
    vector2 = model.wv[word2]
    vector2 = vector2.reshape(1,100)
    cos_sim = cosine_similarity(vector1, vector2)
    print('Cosine similarity from word2vec for {} and {} = {}'.format(word1,word2,cos_sim))
    simlex_out_word2vec.loc[simlex_out_word2vec.index == num,'your_Similarity'] = cos_sim
  except:
    print('skipping the pair {} and {}'.format(word1,word2))



In [0]:
simlex_out_word2vec.to_csv('word2vec.csv',sep='\t')
files.download('word2vec.csv')

## Embedding method 1

In [37]:

simlex_out_USE = copy.deepcopy(simlex_out)
for num in np.arange(0,50,1):
  printbold('word pair {}'.format(num))
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  try:
    print('Embedding this pair from Simlex_999: "{}" and "{}"'.format(word1, word2))
    print('Running universal sentence encoder on "{}"'.format(word1))
    messages = [word1]
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))

      for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        #print("Message: {}".format(messages[i]))
        #print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))

    vector1 = message_embedding
    vector1 = np.asarray(vector1)
    vector1 = vector1.reshape(1,512)

    print('Running universal sentence encoder on "{}"'.format(word2))
    messages = [word2]

    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))
      
      for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        #print("Message: {}".format(messages[i]))
        #print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))

    vector2 = message_embedding
    vector2 = np.asarray(vector2)
    vector2 = vector2.reshape(1,512)

    cos_sim = cosine_similarity(vector1, vector2)
    print('Cosine similarity from Universal sentence encoder for {} and {} = {}\n'.format(word1,word2,cos_sim) )
    simlex_out_USE.loc[simlex_out_USE.index == num,'your_Similarity'] = cos_sim
  except:
    print('skipping the pair {} and {}'.format(word1,word2))




word pair 0
Embedding this pair from Simlex_999: "happy" and "cheerful"
Running universal sentence encoder on "happy"
Running universal sentence encoder on "cheerful"
Cosine similarity from Universal sentence encoder for happy and cheerful = [[0.78390019]]

word pair 1
Embedding this pair from Simlex_999: "bad" and "terrible"
Running universal sentence encoder on "bad"
Running universal sentence encoder on "terrible"
Cosine similarity from Universal sentence encoder for bad and terrible = [[0.86002109]]

word pair 2
Embedding this pair from Simlex_999: "dumb" and "foolish"
Running universal sentence encoder on "dumb"
Running universal sentence encoder on "foolish"
Cosine similarity from Universal sentence encoder for dumb and foolish = [[0.70472723]]

word pair 3
Embedding this pair from Simlex_999: "weird" and "normal"
Running universal sentence encoder on "weird"
Running universal sentence encoder on "normal"
Cosine similarity from Universal sentence encoder for weird and normal = [[

In [0]:
simlex_out_USE_1 = copy.deepcopy(simlex_out_USE)
simlex_out_USE_1 = simlex_out_USE_1.iloc[0:50,:]


In [0]:
simlex_out_USE_1.to_csv('universal_sentence_encoder_1.csv',sep='\t')
files.download('universal_sentence_encoder_1.csv')

In [30]:

simlex_out_USE = copy.deepcopy(simlex_out)
for num in np.arange(50,100,1):
  printbold('word pair {}'.format(num))
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  try:
    print('Embedding this pair from Simlex_999: "{}" and "{}"'.format(word1, word2))
    print('Running universal sentence encoder on "{}"'.format(word1))
    messages = [word1]
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))

      for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        #print("Message: {}".format(messages[i]))
        #print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))

    vector1 = message_embedding
    vector1 = np.asarray(vector1)
    vector1 = vector1.reshape(1,512)

    print('Running universal sentence encoder on "{}"'.format(word2))
    messages = [word2]

    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))
      
      for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        #print("Message: {}".format(messages[i]))
        #print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))

    vector2 = message_embedding
    vector2 = np.asarray(vector2)
    vector2 = vector2.reshape(1,512)

    cos_sim = cosine_similarity(vector1, vector2)
    print('Cosine similarity from Universal sentence encoder for {} and {} = {}\n'.format(word1,word2,cos_sim) )
    simlex_out_USE.loc[simlex_out_USE.index == num,'your_Similarity'] = cos_sim
  except:
    print('skipping the pair {} and {}'.format(word1,word2))




word pair 50
Embedding this pair from Simlex_999: "communication" and "conversation"
Running universal sentence encoder on "communication"
Running universal sentence encoder on "conversation"
Cosine similarity from Universal sentence encoder for communication and conversation = [[0.58857998]]

word pair 51
Embedding this pair from Simlex_999: "college" and "class"
Running universal sentence encoder on "college"
Running universal sentence encoder on "class"
Cosine similarity from Universal sentence encoder for college and class = [[0.54269196]]

word pair 52
Embedding this pair from Simlex_999: "whiskey" and "champagne"
Running universal sentence encoder on "whiskey"
Running universal sentence encoder on "champagne"
Cosine similarity from Universal sentence encoder for whiskey and champagne = [[0.68791156]]

word pair 53
Embedding this pair from Simlex_999: "head" and "nail"
Running universal sentence encoder on "head"
Running universal sentence encoder on "nail"
Cosine similarity from 

In [0]:
simlex_out_USE_2 = copy.deepcopy(simlex_out_USE)
simlex_out_USE_2 = simlex_out_USE_2.iloc[50:100,:]
simlex_out_USE_2.to_csv('universal_sentence_encoder_2.csv',sep='\t')
files.download('universal_sentence_encoder_2.csv')